In [1]:
import requests

res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')

with open('nsmc_train.csv', 'wb') as f:
    f.write(res.content)

In [2]:
import pandas as pd
nsmc=pd.read_csv('nsmc_train.csv', sep='\t')

In [3]:
nsmc.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
from kiwipiepy import Kiwi

In [5]:
kiwi=Kiwi()
kiwi.prepare()

0

In [6]:
text=nsmc.document[0]

In [10]:
text

'아 더빙.. 진짜 짜증나네요 목소리'

In [8]:
def extract_keywords(text):
    result=kiwi.analyze(text)
    for token, pos, start, end in result[0][0]:
        if pos[0] in 'NV':
            yield f'{token}/{pos}'

In [9]:
list(extract_keywords(text))

['더빙/NNG', '짜증/NNG', '나/VV', '목소리/NNG']

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [10]:
nsmc.shape

(150000, 3)

In [11]:
cv=CountVectorizer(max_features=1000, tokenizer=extract_keywords)
dtm=cv.fit_transform(nsmc.loc[0:1999,'document'])

In [12]:
trans = TfidfTransformer()
dtm2=trans.fit_transform(dtm)

In [13]:
words= cv.get_feature_names()

In [14]:
import joblib
joblib.dump({'words':words,'dtm':dtm,'dtm2':dtm2},'nsmc.pkl')

['nsmc.pkl']

In [15]:
data=joblib.load('nsmc.pkl')

In [16]:
locals().update(data)

In [17]:
words

['13구역/NNP',
 '가/VV',
 '가/VX',
 '가능/NNG',
 '가볍/VA',
 '가슴/NNG',
 '가족/NNG',
 '가지/NNB',
 '가지/VV',
 '가치/NNG',
 '각본/NNG',
 '간/NNB',
 '간/NNG',
 '간만/NNG',
 '갈등/NNG',
 '감/NNG',
 '감독/NNG',
 '감동/NNG',
 '감사/NNG',
 '감성/NNG',
 '감정/NNG',
 '강/NNP',
 '강요/NNG',
 '같/VA',
 '개/NNB',
 '개/NNG',
 '개그맨/NNG',
 '개념/NNG',
 '개봉/NNG',
 '개뿔/NNG',
 '개연/NNG',
 '개인/NNG',
 '개판/NNG',
 '걍/NNG',
 '거/NNB',
 '거기/NP',
 '거부/NNG',
 '거슬리/VV',
 '거장/NNG',
 '걱정/NNG',
 '걸/VV',
 '걸리/VV',
 '걸작/NNG',
 '검술/NNG',
 '겁나/VV',
 '것/NNB',
 '게/NNG',
 '게임/NNG',
 '결국/NNG',
 '결말/NNG',
 '고추/NNG',
 '곳/NNG',
 '곳곳/NNG',
 '공감/NNG',
 '공부/NNG',
 '공포/NNG',
 '곽지민/NNP',
 '관/NNG',
 '관객/NNG',
 '관계/NNG',
 '관람/NNG',
 '관심/NNG',
 '괜찮/VA',
 '괜찮/VV',
 '교훈/NNG',
 '구성/NNG',
 '구하/VV',
 '국산/NNG',
 '군/NNG',
 '군대/NNG',
 '굿/NNG',
 '귀신/NNG',
 '귀엽/VA',
 '그/NP',
 '그거/NP',
 '그것/NP',
 '그녀/NP',
 '그때/NNG',
 '그래픽/NNG',
 '그러/VV',
 '그렇/VA',
 '그리/VV',
 '그림/NNG',
 '그립/VA',
 '극/NNG',
 '극장/NNG',
 '급/NNG',
 '기/NNG',
 '기대/NNG',
 '기독교/NNG',
 '기분/NNG',
 '기억/NNG',
 '기존/NNG',
 '긴장/NNG',
 '긴

In [18]:
dtm.shape

(2000, 1000)

In [19]:
x=dtm

In [20]:
y= nsmc.label.values[:2000]

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=1984)

In [23]:
import tensorflow as tf

In [24]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [25]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
#optimizer: 모델을 어떻게 학습 시킬것인가(경사하강법), loss: 손실함수(교차엔트로피)

In [26]:
model.fit(x_train.A ,y_train, epochs=3) #x_train이 압축되어있어 풀어주는 것, epochs 는 몇번 훑냐는 것

Epoch 1/3
50/50 [==============================] - 0s 680us/step - loss: 0.6915 - accuracy: 0.5225
Epoch 2/3
50/50 [==============================] - 0s 760us/step - loss: 0.6748 - accuracy: 0.6313
Epoch 3/3
50/50 [==============================] - 0s 640us/step - loss: 0.6595 - accuracy: 0.6994


In [27]:
model.evaluate(x_test.A,y_test)

13/13 [==============================] - 0s 615us/step - loss: 0.6696 - accuracy: 0.6275


[0.669646143913269, 0.6274999976158142]

In [28]:
model.save('nsmc.krs')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: nsmc.krs\assets


In [29]:
#model=tf.keras.models.load_model('nsmc.krs') 

In [30]:
w,b=model.weights
#w 가 양수면 그 단어가 많이 나올 수 록 긍정적인 단어인 것.

In [38]:
word_sent=pd.DataFrame({'토큰':words,'가중치':w.numpy().flat})

In [36]:
word_sent.sort_values('가중치').head(10)

,토큰,가중치
288,망하/VV,-0.122903
914,터/NNB,-0.120135
537,아깝/VA,-0.117797
808,주인공/NNG,-0.111334
750,잠/NNG,-0.108993
46,게/NNG,-0.108627
662,욕하/VV,-0.107924
28,개봉/NNG,-0.105615
309,목소리/NNG,-0.105091
745,작/NNG,-0.104974


In [33]:
word_sent.sort_values('가중치').tail(10)

,토큰,가중치
219,두/VX,0.109495
854,쩔/VV,0.109810
601,여운/NNG,0.111272
713,인상/NNG,0.113355
748,작품/NNG,0.115054
192,대박/NNG,0.115186
757,재미있/VA,0.130967
801,좋/VA,0.131747
759,재밌/VA,0.143460
881,최고/NNG,0.164953


데이터가 큰 경우 압축을 풀지 않고 텐서플로의 희소행렬로 바꿔서 사용하면 된다.

In [85]:
x_coo=x_train.tocoo()

In [86]:
x_coo.row

array([   0,    0,    0, ..., 1599, 1599, 1599])

In [87]:
x_coo.col

array([  1, 119, 140, ..., 754, 951, 952])

In [88]:
x_coo.data

array([1, 1, 1, ..., 1, 1, 7], dtype=int64)

In [89]:
import numpy as np

In [90]:
index=np.column_stack([x_coo.row,x_coo.col])

In [91]:
index

array([[   0,    1],
       [   0,  119],
       [   0,  140],
       ...,
       [1599,  754],
       [1599,  951],
       [1599,  952]])

In [92]:
x_train_sparse=tf.SparseTensor(index, x_coo.data, x_coo.shape)

In [93]:
x_train_sparse=tf.sparse.reorder(x_train_sparse)

In [94]:
model.fit(x_train_sparse, y_train,epochs=1)

50/50 [==============================] - 0s 940us/step - loss: 0.6397 - accuracy: 0.7494


In [99]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [100]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [101]:
model.fit(x_train_sparse, y_train, epochs=100, validation_split=0.1, 
         callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy')])
#callbacks 특정한 조건이나 상황이 되었을 때 넘겨주라는 것

Epoch 1/100
45/45 [==============================] - 0s 3ms/step - loss: 0.6917 - accuracy: 0.5132 - val_loss: 0.6842 - val_accuracy: 0.6000
Epoch 2/100
45/45 [==============================] - 0s 867us/step - loss: 0.6754 - accuracy: 0.6153 - val_loss: 0.6718 - val_accuracy: 0.6625
Epoch 3/100
45/45 [==============================] - 0s 911us/step - loss: 0.6608 - accuracy: 0.6764 - val_loss: 0.6606 - val_accuracy: 0.7250
Epoch 4/100
45/45 [==============================] - 0s 911us/step - loss: 0.6477 - accuracy: 0.7167 - val_loss: 0.6503 - val_accuracy: 0.7688
Epoch 5/100
45/45 [==============================] - 0s 911us/step - loss: 0.6349 - accuracy: 0.7472 - val_loss: 0.6410 - val_accuracy: 0.7750
Epoch 6/100
45/45 [==============================] - 0s 889us/step - loss: 0.6234 - accuracy: 0.7729 - val_loss: 0.6321 - val_accuracy: 0.7688
